In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_utils, portfolio_utils

import sys
sys.path.append('./scripts/')

from calitp_data_analysis import geography_utils

In [3]:
from update_vars import (analysis_date, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD,
                        HALF_MILE_BUFFER_METERS, INTERSECTION_BUFFER_METERS, HQTA_SEGMENT_LENGTH)

In [4]:
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [5]:
import intake
catalog = intake.open_catalog("*.yml")

# Debug

* solved, was missing a filtering step to only consider azimuth-identified intersections
* next steps are to create "Major Transit Stop/High Quality Transit Corridor Explorer"

In [6]:
ms_spatial = catalog.major_stop_bus.read()

In [7]:
ms_branching = catalog.major_stop_bus_branching.read()

In [10]:
# ms_branching.explore()

In [11]:
# ms_spatial.explore()

In [13]:
gdf = catalog.hqta_points.read()

In [17]:
# gdf.query('hqta_type.str.contains("major")').explore()

In [14]:
# catalog.hqta_points.read().explore()

In [ ]:
path = f"{GCS_FILE_PATH}subset_corridors.parquet"

In [ ]:
corridors = gcsgp.read_parquet(path)

In [ ]:
corridors.circuitous_segment.value_counts()

In [ ]:
m2 = corridors.explore(column='fwd_azimuth_360')

In [ ]:
# ms_spatial.explore(color='red', m=m2)

In [ ]:
pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop.parquet")

In [ ]:
gcsgp.read_parquet(f"{GCS_FILE_PATH}all_bus.parquet")